In [2]:
import numpy as np

In [8]:
class net:
    def __init__(self, layer_dims):
        self.depth = len(layer_dims-1)
        self.W = []
        self.B= []
        self.A = []
        for i in range(self.depth):
            w = np.random.randn(1, layer_dims[i])
            self.W.append(w)
        for i in range(self.depth):
            b = np.zeros((layer_dims[i+1], 1))
            self.B.append(b)
    def train(self, X, Y, epoch, alpha, active_function):
        
    def initialize(self, X):
        m = X.shape[1]
        for i in range(self.depth+1):
            a = np.zeros((layer_dims[i], m))
            self.A.append(a)
    def forward(self, X):
        assert(self.A[0].shape[0] == X.shape[0])
        self.A[0] = X
        for i in range(self.depth):
            self.A[i+1] = self.W[i] @ self.A[i] + self.B[i]
    def backpropogate(self, Y):
        assert(self.A[-1].shape[0] == Y.shape[0])
    
    @static
    def sigmoid(z):
        a = 1/(1+np.exp(-z))
        return a
    
    @static
    def ReLU(z):
        a = np.maximum(0, z)
        return a
    
    @static
    def sigmoidback(a):
        return a * (1-a)
    
    @static
    def ReLUback(a):
        return (a != 0).astype(np.int)

IndentationError: expected an indented block (<ipython-input-8-0d3af2fcbf7f>, line 15)

In [7]:
print(np.array([True,True], dtype=int))

[1 1]
